# AKS 클러스터 실습 가이드

> **실습 환경**: 이 노트북은 Dev Container(GitHub Codespaces 또는 VS Code)에서 실행하도록 구성되어 있습니다. Azure CLI, kubectl 등 필요한 도구가 자동으로 설치되어 있습니다.

> **사전 준비**: Azure 구독과 이전 실습에서 생성한 ACR이 필요합니다.

이 노트북에서는 Azure Kubernetes Service(AKS) 클러스터 생성, 쿠버네티스 및 AKS의 기본 개념, 컨테이너 이미지 배포, 실행, 확장, 모니터링 실습을 진행합니다.

## 1. 쿠버네티스와 AKS 기본 개념 및 이점

### 쿠버네티스란?
쿠버네티스(Kubernetes)는 컨테이너화된 애플리케이션의 배포, 확장, 관리를 자동화하는 오픈소스 플랫폼입니다.

#### 쿠버네티스의 주요 이점
- **자동화된 배포 및 확장**
- **자가 치유(Self-healing)**
- **서비스 디스커버리 및 로드밸런싱**
- **롤링 업데이트 및 롤백**

### AKS란?
AKS(Azure Kubernetes Service)는 Azure에서 제공하는 관리형 쿠버네티스 서비스로, 클러스터 관리의 복잡성을 줄여줍니다.

#### AKS의 주요 이점
- **간편한 클러스터 관리**
- **Azure와의 통합**
- **자동 업그레이드 및 패치**

## 2. AKS 클러스터 생성 실습

Azure CLI를 사용하여 AKS 클러스터를 생성합니다.

> **GitHub Codespaces**: Azure CLI가 이미 설치되어 있으므로 바로 사용 가능합니다.

### 2.1 Azure 로그인

먼저 Azure에 로그인합니다.

In [30]:
# 공통 설정 파일 import
from config import *

# 현재 설정 확인
print_config()

📋 AKS Mini Labs - 현재 설정

🏢 Azure 리소스:
   리소스 그룹: aks-mini-labs-rg
   위치: koreacentral
   AKS 클러스터: aks-mini-labs-cluster
   노드 수: 2
   ACR 이름: (아직 생성되지 않음)

📦 애플리케이션:
   앱 이름: myapp
   이미지 태그: latest
⚠️  경고: ACR_NAME이 설정되지 않았습니다. config.py 파일을 업데이트하세요.
   전체 이미지: (ACR 생성 후 사용 가능)
   Replica 수: 2

📊 모니터링:
   Monitor 워크스페이스: aks-mini-labs-cluster-monitor
   Grafana: aks-mini-labs-grafana

🎯 HPA 설정:
   최소 Replica: 2
   최대 Replica: 10
   CPU 임계값: 50%



In [31]:
%%bash
# Azure 로그인 세션 확인
if az account show &>/dev/null; then
  echo "✅ 이미 Azure에 로그인되어 있습니다."
  echo "   현재 구독: $(az account show --query name -o tsv)"
else
  echo "🔐 Azure 로그인이 필요합니다. 로그인을 진행합니다..."
  az login --output none
  
  # 첫 번째 구독을 자동으로 선택
  SUBSCRIPTION_ID=$(az account list --query "[0].id" -o tsv)
  az account set --subscription $SUBSCRIPTION_ID
  
  echo "✅ Azure 로그인 완료"
  echo "   사용 중인 구독: $(az account show --query name -o tsv)"
fi

✅ 이미 Azure에 로그인되어 있습니다.


/Users/junwoojeong/.azure/cliextensions/ai-examples/azext_ai_examples/custom.py:11: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import parse_version
_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import parse_version


   현재 구독: Visual Studio Enterprise Subscription


### 2.2 리소스 그룹 확인

config.py에서 불러온 변수를 사용하여 리소스 그룹 존재 여부를 확인합니다.

In [32]:
import subprocess

print(f"📝 config.py에서 불러온 변수:")
print(f"   리소스 그룹: {RESOURCE_GROUP}")
print(f"   AKS 클러스터: {AKS_CLUSTER_NAME}")
print()

# 리소스 그룹 존재 확인
result = subprocess.run(
    ["az", "group", "show", "--name", RESOURCE_GROUP],
    capture_output=True,
    text=True
)

if result.returncode == 0:
    print(f"✅ 리소스 그룹이 존재합니다: {RESOURCE_GROUP}")
    subprocess.run(
        ["az", "group", "show", "--name", RESOURCE_GROUP, 
         "--query", "{Name:name, Location:location}", "--output", "table"]
    )
else:
    print(f"❌ 리소스 그룹이 존재하지 않습니다: {RESOURCE_GROUP}")
    print("   1번 노트북을 먼저 실행해주세요.")

📝 config.py에서 불러온 변수:
   리소스 그룹: aks-mini-labs-rg
   AKS 클러스터: aks-mini-labs-cluster

✅ 리소스 그룹이 존재합니다: aks-mini-labs-rg
✅ 리소스 그룹이 존재합니다: aks-mini-labs-rg


/Users/junwoojeong/.azure/cliextensions/ai-examples/azext_ai_examples/custom.py:11: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import parse_version


Name              Location
----------------  ------------
aks-mini-labs-rg  koreacentral


### 2.3 AKS 클러스터 생성

AKS 클러스터 생성 시 Azure Monitor의 관리형 Prometheus와 Grafana를 함께 활성화합니다.

> **참고**: 클러스터 생성에는 약 5-10분 정도 소요됩니다.

In [33]:
import subprocess

# 위 셀에서 설정한 변수 사용
print("🚀 AKS 클러스터 생성을 시작합니다...")
print("⏱️  예상 소요 시간: 5-10분")
print()
print(f"   리소스 그룹: {RESOURCE_GROUP}")
print(f"   클러스터 이름: {AKS_CLUSTER_NAME}")
print()

# 관리형 Prometheus 및 Grafana가 활성화된 AKS 클러스터 생성
result = subprocess.run([
    "az", "aks", "create",
    "--resource-group", RESOURCE_GROUP,
    "--name", AKS_CLUSTER_NAME,
    "--node-count", "2",
    "--enable-managed-identity",
    "--generate-ssh-keys",
    "--verbose"
])

if result.returncode == 0:
    print()
    print("✅ 클러스터 생성 완료!")
    print()
    
    # 클러스터 생성 상태 확인
    print("📋 클러스터 목록:")
    subprocess.run(["az", "aks", "list", "--resource-group", RESOURCE_GROUP, "--output", "table"])
    
    print()
    print("📊 클러스터 상세 정보:")
    subprocess.run(["az", "aks", "show", "--resource-group", RESOURCE_GROUP, 
                    "--name", AKS_CLUSTER_NAME, "--output", "table"])
else:
    print("❌ 클러스터 생성 실패")

🚀 AKS 클러스터 생성을 시작합니다...
⏱️  예상 소요 시간: 5-10분

   리소스 그룹: aks-mini-labs-rg
   클러스터 이름: aks-mini-labs-cluster



/Users/junwoojeong/.azure/cliextensions/ai-examples/azext_ai_examples/custom.py:11: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import parse_version
INFO: Use existing SSH public key file: /Users/junwoojeong/.ssh/id_rsa.pub
INFO: Use existing SSH public key file: /Users/junwoojeong/.ssh/id_rsa.pub


{
  "aadProfile": null,
  "addonProfiles": null,
  "agentPoolProfiles": [
    {
      "artifactStreamingProfile": null,
      "availabilityZones": null,
      "capacityReservationGroupId": null,
      "count": 2,
      "creationData": null,
      "currentOrchestratorVersion": "1.32.7",
      "eTag": "7f7be9ea-812b-4bcf-8b68-db3e2705b373",
      "enableAutoScaling": false,
      "enableEncryptionAtHost": false,
      "enableFips": false,
      "enableNodePublicIp": false,
      "enableUltraSsd": false,
      "gatewayProfile": null,
      "gpuInstanceProfile": null,
      "gpuProfile": null,
      "hostGroupId": null,
      "kubeletConfig": null,
      "kubeletDiskType": "OS",
      "linuxOsConfig": null,
      "localDnsProfile": null,
      "maxCount": null,
      "maxPods": 250,
      "messageOfTheDay": null,
      "minCount": null,
      "mode": "System",
      "name": "nodepool1",
      "networkProfile": {
        "allowedHostPorts": null,
        "applicationSecurityGroups": null,
 

INFO: Command ran in 221.346 seconds (init: 0.131, invoke: 221.215)
/Users/junwoojeong/.azure/cliextensions/ai-examples/azext_ai_examples/custom.py:11: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import parse_version
/Users/junwoojeong/.azure/cliextensions/ai-examples/azext_ai_examples/custom.py:11: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import parse_version


Name                   Location      ResourceGroup     KubernetesVersion    CurrentKubernetesVersion    ProvisioningState    Fqdn
---------------------  ------------  ----------------  -------------------  --------------------------  -------------------  ----------------------------------------------------------------------
aks-mini-labs-cluster  koreacentral  aks-mini-labs-rg  1.32                 1.32.7                      Succeeded            aks-mini-l-aks-mini-labs-rg-8627ae-2sr83kcu.hcp.koreacentral.azmk8s.io

📊 클러스터 상세 정보:


/Users/junwoojeong/.azure/cliextensions/ai-examples/azext_ai_examples/custom.py:11: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import parse_version


Name                   Location      ResourceGroup     KubernetesVersion    CurrentKubernetesVersion    ProvisioningState    Fqdn
---------------------  ------------  ----------------  -------------------  --------------------------  -------------------  ----------------------------------------------------------------------
aks-mini-labs-cluster  koreacentral  aks-mini-labs-rg  1.32                 1.32.7                      Succeeded            aks-mini-l-aks-mini-labs-rg-8627ae-2sr83kcu.hcp.koreacentral.azmk8s.io


### 2.4 클러스터 인증 정보 가져오기

kubectl을 사용하기 위해 AKS 클러스터의 인증 정보를 가져옵니다.

> **GitHub Codespaces**: kubectl이 이미 설치되어 있습니다.

In [34]:
import subprocess

# 위 셀에서 설정한 변수 사용
print(f"🔑 클러스터 인증 정보 가져오기: {AKS_CLUSTER_NAME}")
print()

# 기존 컨텍스트 삭제 (있을 경우)
print("🗑️ 기존 컨텍스트 삭제 중...")
subprocess.run(["kubectl", "config", "delete-context", AKS_CLUSTER_NAME], 
               capture_output=True, text=True)

# 기존 클러스터 삭제 (있을 경우)
subprocess.run(["kubectl", "config", "delete-cluster", AKS_CLUSTER_NAME], 
               capture_output=True, text=True)

print()
print("📥 새로운 클러스터 인증 정보 가져오기...")
# 클러스터 인증 정보 가져오기 (--overwrite-existing 옵션 추가)
subprocess.run(["az", "aks", "get-credentials", 
                "--resource-group", RESOURCE_GROUP, 
                "--name", AKS_CLUSTER_NAME,
                "--overwrite-existing"])

print()
print("📊 kubectl 연결 확인:")
subprocess.run(["kubectl", "get", "nodes"])

print()
print("ℹ️ 클러스터 정보:")
subprocess.run(["kubectl", "cluster-info"])

🔑 클러스터 인증 정보 가져오기: aks-mini-labs-cluster

🗑️ 기존 컨텍스트 삭제 중...

📥 새로운 클러스터 인증 정보 가져오기...


/Users/junwoojeong/.azure/cliextensions/ai-examples/azext_ai_examples/custom.py:11: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import parse_version



📊 kubectl 연결 확인:
NAME                                STATUS   ROLES    AGE   VERSION
aks-nodepool1-24238682-vmss000000   Ready    <none>   65s   v1.32.7
aks-nodepool1-24238682-vmss000001   Ready    <none>   60s   v1.32.7

ℹ️ 클러스터 정보:
Kubernetes control plane is running at https://aks-mini-l-aks-mini-labs-rg-8627ae-2sr83kcu.hcp.koreacentral.azmk8s.io:443
CoreDNS is running at https://aks-mini-l-aks-mini-labs-rg-8627ae-2sr83kcu.hcp.koreacentral.azmk8s.io:443/api/v1/namespaces/kube-system/services/kube-dns:dns/proxy
Metrics-server is running at https://aks-mini-l-aks-mini-labs-rg-8627ae-2sr83kcu.hcp.koreacentral.azmk8s.io:443/api/v1/namespaces/kube-system/services/https:metrics-server:/proxy

To further debug and diagnose cluster problems, use 'kubectl cluster-info dump'.


CompletedProcess(args=['kubectl', 'cluster-info'], returncode=0)

## 3. 컨테이너 이미지 AKS에 배포하기

앞서 ACR에 푸시한 이미지를 AKS에 배포합니다.

### 3.1 ACR을 AKS에 연결

AKS 클러스터가 ACR에 접근할 수 있도록 권한을 부여합니다.

In [35]:
import subprocess

# config.py에서 가져온 ACR_NAME 사용
print("🔍 ACR 정보 확인 중...")
print(f"   config.py에서 가져온 ACR 이름: {ACR_NAME}")
print()

if ACR_NAME is None:
    # config.py에 ACR_NAME이 설정되지 않은 경우, 리소스 그룹에서 자동 조회
    print("⚠️  config.py에 ACR_NAME이 설정되지 않았습니다.")
    print("   리소스 그룹에서 ACR을 자동으로 조회합니다...")
    print()
    acr_result = subprocess.run(
        ["az", "acr", "list", "--resource-group", RESOURCE_GROUP, "--query", "[0].name", "-o", "tsv"],
        capture_output=True,
        text=True
    )
    
    if acr_result.returncode == 0 and acr_result.stdout.strip():
        ACR_NAME = acr_result.stdout.strip()
        print(f"✅ ACR 발견: {ACR_NAME}")
        print()
        print("💡 권장 사항:")
        print(f"   config.py 파일을 열고 ACR_NAME = \"{ACR_NAME}\"로 업데이트하세요.")
    else:
        print("❌ ACR을 찾을 수 없습니다. 1번 노트북을 먼저 실행해주세요.")
        raise Exception("ACR not found")
else:
    print(f"✅ ACR 이름: {ACR_NAME}")

print()
print(f"🔗 ACR을 AKS에 연결 중...")
print(f"   ACR: {ACR_NAME}")
print(f"   AKS: {AKS_CLUSTER_NAME}")
print()

# ACR을 AKS에 연결 (attach-acr)
# 이 명령이 자동으로 AcrPull 권한을 kubelet identity에 부여합니다

result = subprocess.run([
    "az", "aks", "update",
    "--resource-group", RESOURCE_GROUP,
    "--name", AKS_CLUSTER_NAME,
    "--attach-acr", ACR_NAME
])

if result.returncode == 0:
    print("✅ ACR 연결 완료!")
    print()
    print("   AKS가 ACR에서 이미지를 pull할 수 있는 권한이 자동으로 설정되었습니다.")
    print()
    
    # 클러스터가 ACR 권한을 인식할 때까지 대기
    print("⏱️  클러스터가 ACR 권한을 인식할 때까지 30초 대기 중...")
    import time
    for i in range(30, 0, -5):
        print(f"   {i}초 남음...", end='\r')
        time.sleep(5)
    print("   ✓ 대기 완료!" + " " * 20)
    print()
    
    # 클러스터 상태 확인
    print("📊 클러스터 상태 확인:")
    subprocess.run(["az", "aks", "show", 
                    "--resource-group", RESOURCE_GROUP, 
                    "--name", AKS_CLUSTER_NAME,
                    "--query", "{provisioningState:provisioningState, powerState:powerState.code, nodeResourceGroup:nodeResourceGroup}",
                    "--output", "table"])
    print()
    
    # ACR 연결 상태 확인
    print("🔍 ACR 연결 상태 확인:")
    acr_id_result = subprocess.run(
        ["az", "acr", "show", "--name", ACR_NAME, "--query", "id", "-o", "tsv"],
        capture_output=True,
        text=True
    )
    
    if acr_id_result.returncode == 0:
        acr_id = acr_id_result.stdout.strip()
        print(f"   ACR ID: {acr_id}")
        
        # kubelet identity의 역할 할당 확인
        print("   Kubelet identity의 AcrPull 권한 확인 중...")
        role_result = subprocess.run(
            ["az", "role", "assignment", "list",
             "--scope", acr_id,
             "--query", "[?roleDefinitionName=='AcrPull'].{Principal:principalName, Role:roleDefinitionName, Scope:scope}",
             "--output", "table"],
            capture_output=True,
            text=True
        )
        
        if role_result.returncode == 0 and role_result.stdout.strip():
            print("   ✅ AcrPull 권한이 정상적으로 할당되었습니다.")
            print()
            print(role_result.stdout)
        else:
            print("   ⚠️  권한 확인 중 오류 발생 (하지만 정상적으로 작동할 수 있습니다)")
    print()
else:
    print("❌ ACR 연결 실패")

print()

🔍 ACR 정보 확인 중...
   config.py에서 가져온 ACR 이름: None

⚠️  config.py에 ACR_NAME이 설정되지 않았습니다.
   리소스 그룹에서 ACR을 자동으로 조회합니다...

✅ ACR 발견: myacr1760426545

💡 권장 사항:
   config.py 파일을 열고 ACR_NAME = "myacr1760426545"로 업데이트하세요.

🔗 ACR을 AKS에 연결 중...
   ACR: myacr1760426545
   AKS: aks-mini-labs-cluster

✅ ACR 발견: myacr1760426545

💡 권장 사항:
   config.py 파일을 열고 ACR_NAME = "myacr1760426545"로 업데이트하세요.

🔗 ACR을 AKS에 연결 중...
   ACR: myacr1760426545
   AKS: aks-mini-labs-cluster



/Users/junwoojeong/.azure/cliextensions/ai-examples/azext_ai_examples/custom.py:11: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import parse_version
AAD role propagation done[############################################]  100.0000%WARNING: keyvaultid: None, enable_kv: None


{
  "aadProfile": null,
  "addonProfiles": null,
  "agentPoolProfiles": [
    {
      "artifactStreamingProfile": null,
      "availabilityZones": null,
      "capacityReservationGroupId": null,
      "count": 2,
      "creationData": null,
      "currentOrchestratorVersion": "1.32.7",
      "eTag": "91a72c07-fc00-4851-92b2-37572a27c8e4",
      "enableAutoScaling": false,
      "enableEncryptionAtHost": false,
      "enableFips": false,
      "enableNodePublicIp": false,
      "enableUltraSsd": false,
      "gatewayProfile": null,
      "gpuInstanceProfile": null,
      "gpuProfile": null,
      "hostGroupId": null,
      "kubeletConfig": null,
      "kubeletDiskType": "OS",
      "linuxOsConfig": null,
      "localDnsProfile": null,
      "maxCount": null,
      "maxPods": 250,
      "messageOfTheDay": null,
      "minCount": null,
      "mode": "System",
      "name": "nodepool1",
      "networkProfile": {
        "allowedHostPorts": null,
        "applicationSecurityGroups": null,
 

/Users/junwoojeong/.azure/cliextensions/ai-examples/azext_ai_examples/custom.py:11: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import parse_version


ProvisioningState    PowerState    NodeResourceGroup
-------------------  ------------  ------------------------------------------------------
Succeeded            Running       MC_aks-mini-labs-rg_aks-mini-labs-cluster_koreacentral

🔍 ACR 연결 상태 확인:
   ACR ID: /subscriptions/8627ae60-01d3-4a2d-9c33-89dea54cd4b4/resourceGroups/aks-mini-labs-rg/providers/Microsoft.ContainerRegistry/registries/myacr1760426545
   Kubelet identity의 AcrPull 권한 확인 중...
   ✅ AcrPull 권한이 정상적으로 할당되었습니다.

Principal                             Role     Scope
------------------------------------  -------  ----------------------------------------------------------------------------------------------------------------------------------------------------
86b1f2dd-8fa0-4660-8811-f3f49954087b  AcrPull  /subscriptions/8627ae60-01d3-4a2d-9c33-89dea54cd4b4/resourceGroups/aks-mini-labs-rg/providers/Microsoft.ContainerRegistry/registries/myacr1760426545





### 3.2 ACR 이미지 확인

배포하기 전에 ACR에 어떤 이미지가 있는지 확인합니다.

In [36]:
import subprocess

print(f"📦 ACR 저장소 목록 확인: {ACR_NAME}")
print()

# ACR 저장소 목록 조회
repo_result = subprocess.run(
    ["az", "acr", "repository", "list", "--name", ACR_NAME, "--output", "table"],
    capture_output=True,
    text=True
)

if repo_result.returncode == 0:
    print(repo_result.stdout)
    
    # 저장소 이름 추출하여 태그 확인
    repos_output = subprocess.run(
        ["az", "acr", "repository", "list", "--name", ACR_NAME, "-o", "tsv"],
        capture_output=True,
        text=True
    )
    
    if repos_output.returncode == 0 and repos_output.stdout.strip():
        repos = repos_output.stdout.strip().split('\n')
        print()
        print("🏷️  각 이미지의 태그 확인:")
        print()
        
        for repo in repos:
            if repo:
                print(f"   📂 저장소: {repo}")
                subprocess.run([
                    "az", "acr", "repository", "show-tags",
                    "--name", ACR_NAME,
                    "--repository", repo,
                    "--output", "table"
                ])
                print()
else:
    print("❌ ACR 저장소 목록 조회 실패")
    print(repo_result.stderr)

print()
print("💡 1번 노트북에서 이미지를 푸시했는지 확인하세요.")
print("   이미지가 없다면 1번 노트북의 섹션 6.7(이미지 푸시)를 실행하세요.")

📦 ACR 저장소 목록 확인: myacr1760426545

Result
--------
myapp

Result
--------
myapp


🏷️  각 이미지의 태그 확인:

   📂 저장소: myapp

🏷️  각 이미지의 태그 확인:

   📂 저장소: myapp


/Users/junwoojeong/.azure/cliextensions/ai-examples/azext_ai_examples/custom.py:11: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import parse_version


Result
--------
latest


💡 1번 노트북에서 이미지를 푸시했는지 확인하세요.
   이미지가 없다면 1번 노트북의 섹션 6.7(이미지 푸시)를 실행하세요.


### 3.3 Deployment YAML 파일 작성

`deployment.yaml` 파일을 자동으로 생성합니다.

In [38]:
# config.py에서 가져온 변수 사용하여 deployment.yaml 생성
deployment_yaml = f"""apiVersion: apps/v1
kind: Deployment
metadata:
  name: {DEPLOYMENT_NAME}
spec:
  replicas: {REPLICAS}
  selector:
    matchLabels:
      app: {APP_NAME}
  template:
    metadata:
      labels:
        app: {APP_NAME}
    spec:
      containers:
      - name: {APP_NAME}
        image: {get_full_image_name(ACR_NAME, APP_NAME, IMAGE_TAG)}
        ports:
        - containerPort: 8080
        resources:
          requests:
            cpu: 100m
            memory: 128Mi
          limits:
            cpu: 250m
            memory: 256Mi
"""

# 파일 생성
with open("deployment.yaml", "w") as f:
    f.write(deployment_yaml)

print(f"✅ deployment.yaml 파일 생성 완료")
print(f"   이미지: {get_full_image_name(ACR_NAME, APP_NAME, IMAGE_TAG)}")
print()
print("💡 사용된 config.py 변수:")
print(f"   DEPLOYMENT_NAME: {DEPLOYMENT_NAME}")
print(f"   APP_NAME: {APP_NAME}")
print(f"   REPLICAS: {REPLICAS}")
print(f"   ACR_NAME: {ACR_NAME}")
print()
print("📄 생성된 파일 내용:")
print()
print(deployment_yaml)

✅ deployment.yaml 파일 생성 완료
   이미지: myacr1760426545.azurecr.io/myapp:latest

💡 사용된 config.py 변수:
   DEPLOYMENT_NAME: myapp
   APP_NAME: myapp
   REPLICAS: 2
   ACR_NAME: myacr1760426545

📄 생성된 파일 내용:

apiVersion: apps/v1
kind: Deployment
metadata:
  name: myapp
spec:
  replicas: 2
  selector:
    matchLabels:
      app: myapp
  template:
    metadata:
      labels:
        app: myapp
    spec:
      containers:
      - name: myapp
        image: myacr1760426545.azurecr.io/myapp:latest
        ports:
        - containerPort: 8080
        resources:
          requests:
            cpu: 100m
            memory: 128Mi
          limits:
            cpu: 250m
            memory: 256Mi



### 3.4 Deployment 생성 및 Service 노출

In [39]:
import subprocess

print("📦 Deployment 생성 중...")
subprocess.run(["kubectl", "apply", "-f", "deployment.yaml"])

print()
print("📊 Deployment 상태 확인:")
subprocess.run(["kubectl", "get", "deployments"])

print()
print("🔍 Pod 상태 확인:")
subprocess.run(["kubectl", "get", "pods"])

print()
print("🌐 Service 생성 중 (LoadBalancer)...")
subprocess.run(["kubectl", "expose", "deployment", DEPLOYMENT_NAME, 
                "--type=LoadBalancer", 
                "--port=80", 
                "--target-port=8080"])

print()
print("📋 Service 상태 확인:")
subprocess.run(["kubectl", "get", "services"])

📦 Deployment 생성 중...
deployment.apps/myapp created

📊 Deployment 상태 확인:
NAME    READY   UP-TO-DATE   AVAILABLE   AGE
myapp   0/2     0            0           1s

🔍 Pod 상태 확인:
deployment.apps/myapp created

📊 Deployment 상태 확인:
NAME    READY   UP-TO-DATE   AVAILABLE   AGE
myapp   0/2     0            0           1s

🔍 Pod 상태 확인:
NAME                     READY   STATUS              RESTARTS   AGE
myapp-6c57475dcf-hj24f   0/1     ContainerCreating   0          0s
myapp-6c57475dcf-p8svh   0/1     ContainerCreating   0          0s

🌐 Service 생성 중 (LoadBalancer)...
service/myapp exposed

📋 Service 상태 확인:
NAME                     READY   STATUS              RESTARTS   AGE
myapp-6c57475dcf-hj24f   0/1     ContainerCreating   0          0s
myapp-6c57475dcf-p8svh   0/1     ContainerCreating   0          0s

🌐 Service 생성 중 (LoadBalancer)...
service/myapp exposed

📋 Service 상태 확인:
NAME         TYPE           CLUSTER-IP     EXTERNAL-IP   PORT(S)        AGE
kubernetes   ClusterIP      10.0.0.1       

CompletedProcess(args=['kubectl', 'get', 'services'], returncode=0)

### 3.5 외부 IP 확인 및 애플리케이션 접근

LoadBalancer 서비스가 생성되면 Azure가 자동으로 공용 IP를 할당합니다. 이 IP를 통해 애플리케이션에 접근할 수 있습니다.

> **참고**: 외부 IP 할당에는 1-2분 정도 소요될 수 있습니다. `<pending>` 상태에서 실제 IP 주소로 변경될 때까지 기다려주세요.

In [40]:
import subprocess

print("🌐 외부 IP 확인")
print("=" * 80)
print()

print(f"📋 Service 정보 확인: {SERVICE_NAME}")
subprocess.run(["kubectl", "get", "service", SERVICE_NAME])

print()

print("📊 외부 IP 상세 정보:")
result = subprocess.run([
    "kubectl", "get", "service", SERVICE_NAME,
    "-o", "jsonpath={.status.loadBalancer.ingress[0].ip}"
], capture_output=True, text=True)

external_ip = result.stdout.strip()

if external_ip:
    print(f"✅ 외부 IP: {external_ip}")
    print()
    print("🌐 애플리케이션 접근:")
    print(f"   브라우저: http://{external_ip}")
    print(f"   또는 터미널: curl http://{external_ip}")
    print()
    print("💡 애플리케이션이 정상적으로 응답하는지 확인하세요!")
else:
    print("⚠️  외부 IP가 아직 할당되지 않았습니다 (<pending> 상태)")
    print()
    print("💡 다음 단계:")
    print("   1. 1-2분 후 이 셀을 다시 실행하세요")
    print("   2. 또는 다음 명령으로 실시간 모니터링:")
    print(f"      kubectl get service {SERVICE_NAME} --watch")

print()
print("=" * 80)

🌐 외부 IP 확인

📋 Service 정보 확인: myapp
NAME    TYPE           CLUSTER-IP     EXTERNAL-IP    PORT(S)        AGE
myapp   LoadBalancer   10.0.220.129   4.230.112.29   80:32390/TCP   18s

📊 외부 IP 상세 정보:
✅ 외부 IP: 4.230.112.29

🌐 애플리케이션 접근:
   브라우저: http://4.230.112.29
   또는 터미널: curl http://4.230.112.29

💡 애플리케이션이 정상적으로 응답하는지 확인하세요!



## 4. 애플리케이션 확장

Kubernetes에서는 replica 수를 조정하여 애플리케이션을 쉽게 확장할 수 있습니다.

### 4.1 수동 스케일링

In [41]:
import subprocess

print("📈 Replica 수를 4로 확장 중...")
subprocess.run(["kubectl", "scale", "deployment", "myapp", "--replicas=4"])

print()
print("🔍 Pod 상태 확인 (상세):")
subprocess.run(["kubectl", "get", "pods", "-o", "wide"])

print()
print("📊 Deployment 상태:")
subprocess.run(["kubectl", "get", "deployment", "myapp"])

📈 Replica 수를 4로 확장 중...
deployment.apps/myapp scaled

🔍 Pod 상태 확인 (상세):
NAME                     READY   STATUS              RESTARTS   AGE   IP             NODE                                NOMINATED NODE   READINESS GATES
myapp-6c57475dcf-8pfk6   0/1     ContainerCreating   0          0s    <none>         aks-nodepool1-24238682-vmss000001   <none>           <none>
myapp-6c57475dcf-hj24f   1/1     Running             0          26s   10.244.0.49    aks-nodepool1-24238682-vmss000000   <none>           <none>
myapp-6c57475dcf-p8svh   1/1     Running             0          26s   10.244.1.252   aks-nodepool1-24238682-vmss000001   <none>           <none>

📊 Deployment 상태:
NAME    READY   UP-TO-DATE   AVAILABLE   AGE
myapp   2/4     4            2           27s


CompletedProcess(args=['kubectl', 'get', 'deployment', 'myapp'], returncode=0)

### 4.2 HPA(Horizontal Pod Autoscaler)로 오토스케일링 적용

HPA를 사용하면 CPU 사용률 등의 메트릭에 따라 자동으로 Pod 수를 조정할 수 있습니다.

In [ ]:
import subprocess

print("🎯 HPA 생성 중 (CPU 사용률 50% 기준, 최소 2개, 최대 10개)...")
subprocess.run(["kubectl", "autoscale", "deployment", "myapp", 
                "--cpu-percent=50", 
                "--min=2", 
                "--max=10"])

print()
print("📊 HPA 상태:")
subprocess.run(["kubectl", "get", "hpa"])

print()
print("📋 HPA 상세 정보:")
subprocess.run(["kubectl", "describe", "hpa", "myapp"])

### 4.3 HPA 오토스케일링 테스트

HPA가 정상적으로 동작하는지 확인하기 위해 부하를 발생시킵니다.

#### 방법 1: 부하 생성 도구 사용

In [ ]:
import subprocess

print("🔥 부하 생성용 Pod 실행 중...")
subprocess.run(["kubectl", "run", "load-generator", 
                "--image=busybox", 
                "--restart=Never", 
                "--", "/bin/sh", "-c", 
                "while true; do wget -q -O- http://myapp; done"])

print()
print("✅ 부하 생성 Pod가 실행되었습니다.")
print()
print("   HPA 상태를 모니터링하려면 별도 터미널에서 다음 명령 실행:")
print("   kubectl get hpa myapp --watch")
print()
print("   또는 Pod 수 변화 모니터링:")
print("   kubectl get pods --watch")

#### 테스트 정리

부하 테스트가 완료되면 부하 생성 Pod를 삭제합니다.

In [ ]:
import subprocess

print("🧹 부하 생성 Pod 삭제 중...")
subprocess.run(["kubectl", "delete", "pod", "load-generator"])

print()
print("✅ 부하 생성 Pod가 삭제되었습니다.")

### HPA 오토스케일링 테스트 방법
HPA가 정상적으로 동작하는지 확인하려면, 파드에 부하를 인위적으로 발생시켜 오토스케일링이 동작하는지 관찰할 수 있습니다.

1. **stress 도구 설치** (app 파드에 설치):
```bash
kubectl exec -it <app-pod-name> -- apt-get update
kubectl exec -it <app-pod-name> -- apt-get install -y stress
```

2. **CPU 부하 발생**:
```bash
kubectl exec -it <app-pod-name> -- stress --cpu 1 --timeout 300
```

3. **HPA 상태 모니터링**:
```bash
kubectl get hpa -w
```

- 위 과정을 통해 CPU 사용률이 증가하면 HPA가 자동으로 파드 수를 늘리는 것을 확인할 수 있습니다.

 > 참고: `<app-pod-name>`은 실제 app 파드 이름으로 대체하세요. 파드 이름은 `kubectl get pods` 명령으로 확인할 수 있습니다.

## 5. 모니터링 및 관찰성(Observability)

AKS는 Azure Monitor, Prometheus, Grafana, Container Insights와 통합되어 클러스터 및 애플리케이션의 완전한 관찰성을 제공합니다.

**관찰성의 3가지 핵심 요소:**
1. **메트릭(Metrics)**: Prometheus로 실시간 성능 지표 수집
2. **로그(Logs)**: Container Insights로 컨테이너 로그 및 이벤트 수집
3. **시각화(Visualization)**: Grafana로 대시보드를 통한 데이터 시각화

### 5.1 Azure Monitor 기본 모니터링

Azure Portal에서 AKS 리소스를 선택하고, 모니터링 탭에서 다음을 확인할 수 있습니다:
- **Insights**: 클러스터, 노드, 컨트롤러, 컨테이너 상태
- **Metrics**: CPU, Memory, Network 등의 메트릭
- **Logs**: KQL(Kusto Query Language)로 로그 쿼리
- **Workbooks**: 사전 구성된 모니터링 대시보드

### 5.2 관리형 Prometheus, Grafana 및 Container Insights 활성화

AKS 클러스터에 완전한 관찰성(Observability) 스택을 구성합니다:
- **Azure Monitor Managed Prometheus**: 메트릭 수집 (CPU, Memory, Network 등)
- **Azure Managed Grafana**: 시각화 대시보드
- **Container Insights**: 컨테이너 로그 및 이벤트 수집

이 3가지 솔루션을 통해 메트릭, 로그, 시각화를 모두 제공하는 통합 모니터링 환경을 구축합니다.

### 5.2.1 Prometheus & Grafana 설정

모니터링 스택을 두 단계로 나누어 설정합니다:

**Part 1: Prometheus 설정**
- Azure Monitor Workspace 생성 (Prometheus 메트릭 저장소)
- AKS에 Prometheus addon 활성화 (메트릭 수집 에이전트)
- AKS 클러스터의 메트릭을 Azure Monitor Workspace로 전송

**Part 2: Grafana 설정 및 데이터 소스 연결**
- Azure Managed Grafana 생성 (시각화 도구)
- AKS와 Grafana 연결
- **Grafana가 Prometheus의 데이터를 조회하도록 데이터 소스 설정** ⭐

> 💡 **중요**: Part 1에서 Prometheus가 메트릭을 수집하고, Part 2에서 Grafana가 그 메트릭을 읽어서 시각화합니다.

#### Part 1: Prometheus 메트릭 수집 설정

먼저 Prometheus 기반의 메트릭 수집 환경을 구축합니다.

**이 셀에서 수행하는 작업:**
1. **Azure Monitor Workspace 생성**: Prometheus 형식의 메트릭을 저장할 중앙 저장소
2. **Prometheus Query Endpoint 확보**: 나중에 Grafana가 데이터를 조회할 때 사용할 주소
3. **AKS Prometheus Addon 활성화**: AKS 클러스터에서 실행되는 메트릭 수집 에이전트 설치
4. **메트릭 전송 설정**: AKS → Azure Monitor Workspace로 실시간 메트릭 전송

실행 후 AKS 클러스터의 모든 Pod, Node, Container 메트릭이 자동으로 수집되기 시작합니다.

In [42]:
import subprocess
import warnings
import json

# Azure CLI 확장의 pkg_resources 경고 메시지 숨기기
warnings.filterwarnings('ignore', message='.*pkg_resources.*')

print("🔧 Part 1: Prometheus 설정")
print("=" * 80)
print()
print("📊 구성할 Prometheus 스택:")
print("   1. Azure Monitor Workspace (Prometheus 메트릭 저장소)")
print("   2. AKS Prometheus Addon (메트릭 수집 에이전트)")
print("=" * 80)
print()

# config.py에서 가져온 변수 사용
# Azure Monitor 워크스페이스 생성
print("1️⃣ Azure Monitor 워크스페이스 생성")
print(f"   워크스페이스 이름: {MONITOR_WORKSPACE_NAME}")
print(f"   리소스 그룹: {RESOURCE_GROUP}")
print(f"   위치: {LOCATION}")
print()

result = subprocess.run([
    "az", "monitor", "account", "create",
    "--name", MONITOR_WORKSPACE_NAME,
    "--resource-group", RESOURCE_GROUP,
    "--location", LOCATION
], capture_output=True, text=True)

if result.returncode == 0:
    print(f"✅ Azure Monitor 워크스페이스 생성 완료: {MONITOR_WORKSPACE_NAME}")
else:
    print("⚠️ 워크스페이스가 이미 존재하거나 생성 중 오류 발생")
    if result.stderr:
        print(f"   {result.stderr}")

print()

# Monitor 워크스페이스 정보 가져오기
print("2️⃣ Azure Monitor 워크스페이스 정보 확인")
ws_result = subprocess.run([
    "az", "monitor", "account", "show",
    "--name", MONITOR_WORKSPACE_NAME,
    "--resource-group", RESOURCE_GROUP,
    "--query", "{id:id, queryEndpoint:metrics.prometheusQueryEndpoint}",
    "-o", "json"
], capture_output=True, text=True)

if ws_result.returncode == 0:
    ws_info = json.loads(ws_result.stdout)
    MONITOR_WORKSPACE_ID = ws_info['id']
    PROMETHEUS_QUERY_ENDPOINT = ws_info.get('queryEndpoint', '')
    print(f"✅ 워크스페이스 ID: {MONITOR_WORKSPACE_ID}")
    print(f"✅ Prometheus Query Endpoint: {PROMETHEUS_QUERY_ENDPOINT}")
else:
    print("❌ 워크스페이스 정보 조회 실패")
    print(ws_result.stderr)
    raise Exception("Azure Monitor Workspace 정보를 가져올 수 없습니다")

print()

# AKS에 Prometheus 애드온 활성화
print("3️⃣ AKS에 Prometheus 애드온 활성화")
print("   ⏳ AKS 클러스터에 메트릭 수집 에이전트 설치 중...")
result = subprocess.run([
    "az", "aks", "update",
    "--resource-group", RESOURCE_GROUP,
    "--name", AKS_CLUSTER_NAME,
    "--enable-azure-monitor-metrics",
    "--azure-monitor-workspace-resource-id", MONITOR_WORKSPACE_ID
], capture_output=True, text=True)

if result.returncode == 0:
    print("✅ Prometheus 애드온 활성화 완료")
    print("   📊 AKS 클러스터 → Azure Monitor Workspace로 메트릭 전송 중")
else:
    print("⚠️ Prometheus 애드온 활성화 중 오류")
    if result.stderr:
        print(f"   {result.stderr}")

print()
print("✅ Part 1 완료: Prometheus 설정 완료")
print("=" * 80)

🔧 Part 1: Prometheus 설정

📊 구성할 Prometheus 스택:
   1. Azure Monitor Workspace (Prometheus 메트릭 저장소)
   2. AKS Prometheus Addon (메트릭 수집 에이전트)

1️⃣ Azure Monitor 워크스페이스 생성
   워크스페이스 이름: aks-mini-labs-cluster-monitor
   리소스 그룹: aks-mini-labs-rg
   위치: koreacentral

✅ Azure Monitor 워크스페이스 생성 완료: aks-mini-labs-cluster-monitor

2️⃣ Azure Monitor 워크스페이스 정보 확인
✅ 워크스페이스 ID: /subscriptions/8627ae60-01d3-4a2d-9c33-89dea54cd4b4/resourcegroups/aks-mini-labs-rg/providers/microsoft.monitor/accounts/aks-mini-labs-cluster-monitor
✅ Prometheus Query Endpoint: https://aks-mini-labs-cluster-monitor-djc8fzdxgxerd2br.koreacentral.prometheus.monitor.azure.com

3️⃣ AKS에 Prometheus 애드온 활성화
   ⏳ AKS 클러스터에 메트릭 수집 에이전트 설치 중...
✅ Prometheus 애드온 활성화 완료
   📊 AKS 클러스터 → Azure Monitor Workspace로 메트릭 전송 중

✅ Part 1 완료: Prometheus 설정 완료


#### Part 2: Grafana 시각화 도구 설정

이제 수집된 Prometheus 메트릭을 시각화할 Grafana를 설정합니다.

**이 셀에서 수행하는 작업:**
1. **Azure Managed Grafana 생성**: 완전 관리형 Grafana 인스턴스 생성
2. **AKS와 Grafana 연결**: AKS가 이 Grafana를 기본 시각화 도구로 사용하도록 설정
3. **데이터 소스 연결 (핵심!)**: 
   - Grafana → Prometheus Query Endpoint 연결
   - Grafana가 Azure Monitor Workspace의 Prometheus 메트릭을 조회할 수 있도록 설정
   - Azure Managed Identity를 사용한 인증 구성

**연결 흐름:**
```
AKS Cluster → [메트릭 수집] → Azure Monitor Workspace (Prometheus)
                                            ↓ [데이터 조회]
                                        Grafana (시각화)
```

실행 후 Grafana에서 Kubernetes 대시보드를 통해 실시간 메트릭을 확인할 수 있습니다.

In [43]:
import subprocess
import warnings
import json
import time

# Azure CLI 확장의 pkg_resources 경고 메시지 숨기기
warnings.filterwarnings('ignore', message='.*pkg_resources.*')

print("🔧 Part 2: Grafana 설정 및 Prometheus 데이터 소스 연결")
print("=" * 80)
print()
print("📊 구성할 Grafana 스택:")
print("   1. Azure Managed Grafana (시각화 도구)")
print("   2. Grafana가 Prometheus 데이터를 조회하도록 데이터 소스 연결 ⭐")
print("=" * 80)
print()

# Grafana 워크스페이스 생성
print("1️⃣ 관리형 Grafana 워크스페이스 생성")
print(f"   Grafana 이름: {GRAFANA_NAME}")
print(f"   리소스 그룹: {RESOURCE_GROUP}")
print()

result = subprocess.run([
    "az", "grafana", "create",
    "--name", GRAFANA_NAME,
    "--resource-group", RESOURCE_GROUP
], capture_output=True, text=True)

if result.returncode == 0:
    print(f"✅ Grafana 워크스페이스 생성 완료: {GRAFANA_NAME}")
else:
    print("⚠️ Grafana가 이미 존재하거나 생성 중 오류 발생")
    if result.stderr:
        print(f"   {result.stderr}")

# Grafana 생성 확인
print()
print("   🔍 Grafana 워크스페이스 상태 확인 중...")
grafana_check = subprocess.run([
    "az", "grafana", "show",
    "--name", GRAFANA_NAME,
    "--resource-group", RESOURCE_GROUP,
    "--query", "{id:id, endpoint:properties.endpoint, provisioningState:properties.provisioningState}",
    "-o", "json"
], capture_output=True, text=True)

if grafana_check.returncode == 0:
    grafana_info = json.loads(grafana_check.stdout)
    GRAFANA_ID = grafana_info['id']
    GRAFANA_ENDPOINT = grafana_info.get('endpoint', '')
    GRAFANA_STATE = grafana_info.get('provisioningState', '')
    
    print(f"   ✅ Grafana ID: {GRAFANA_ID}")
    print(f"   ✅ Grafana URL: {GRAFANA_ENDPOINT}")
    print(f"   ✅ 상태: {GRAFANA_STATE}")
else:
    print("   ❌ Grafana 정보 조회 실패")
    if grafana_check.stderr:
        print(f"   {grafana_check.stderr}")
    raise Exception("Grafana 워크스페이스를 확인할 수 없습니다")

print()

# AKS에 Grafana 연결
print("2️⃣ AKS에 Grafana 연결 설정")
print("   💡 AKS가 이 Grafana를 사용하도록 연결")
print("   💡 중요: Prometheus 메트릭과 함께 설정해야 grafanaResourceId가 적용됩니다!")
print("   ⏳ 설정 중... (2-3분 소요)")
print()

update_result = subprocess.run([
    "az", "aks", "update",
    "--resource-group", RESOURCE_GROUP,
    "--name", AKS_CLUSTER_NAME,
    "--enable-azure-monitor-metrics",
    "--azure-monitor-workspace-resource-id", MONITOR_WORKSPACE_ID,
    "--grafana-resource-id", GRAFANA_ID
], capture_output=True, text=True)

if update_result.returncode == 0:
    # 실제 연결 상태 확인
    print("   ⏳ 연결 상태 확인 중... (10초 대기)")
    time.sleep(10)
    
    check_result = subprocess.run([
        "az", "aks", "show",
        "--name", AKS_CLUSTER_NAME,
        "--resource-group", RESOURCE_GROUP,
        "--query", "azureMonitorProfile.metrics.grafanaResourceId",
        "-o", "tsv"
    ], capture_output=True, text=True)
    
    if check_result.returncode == 0 and check_result.stdout.strip():
        print("   ✅ AKS → Grafana 연결 설정 성공!")
        print(f"   📊 설정된 Grafana: {check_result.stdout.strip()}")
    else:
        print("   ⚠️  설정 명령은 완료되었으나 상태 확인 실패")
        print("   💡 5-10분 후 Portal에서 확인 필요")
else:
    print("   ❌ Grafana 연결 설정 실패:")
    if update_result.stderr:
        print(f"   {update_result.stderr}")
    print()
    print("   💡 해결 방법:")
    print("      1. 5-10분 후 재시도")
    print("      2. Azure Portal → AKS → Configure monitoring에서 수동 설정")

print()

# Grafana에 Prometheus 데이터 소스 추가 (진짜 연결!)
print("3️⃣ Grafana가 Prometheus 데이터를 조회하도록 데이터 소스 연결 ⭐")
print("   💡 이것이 진짜 Prometheus와 Grafana 연결입니다!")
print("   💡 Grafana가 Prometheus 메트릭을 읽을 수 있게 설정")
print()

# 기존 데이터 소스 확인
print("   🔍 기존 Prometheus 데이터 소스 확인 중...")
list_ds_result = subprocess.run([
    "az", "grafana", "data-source", "list",
    "--name", GRAFANA_NAME,
    "--resource-group", RESOURCE_GROUP,
    "-o", "json"
], capture_output=True, text=True)

prometheus_ds_exists = False
if list_ds_result.returncode == 0:
    try:
        datasources = json.loads(list_ds_result.stdout)
        for ds in datasources:
            if ds.get('type') == 'prometheus' and 'azure' in ds.get('name', '').lower():
                prometheus_ds_exists = True
                print(f"   ✅ 기존 Prometheus 데이터 소스 발견: {ds.get('name')}")
                print(f"      URL: {ds.get('url', 'N/A')}")
                break
    except Exception as e:
        print(f"   ⚠️  데이터 소스 목록 파싱 중 오류: {e}")

if not prometheus_ds_exists:
    print("   ⏳ Prometheus 데이터 소스 추가 중...")
    print(f"      Grafana → Azure Monitor Workspace(Prometheus) 연결")
    
    # Prometheus 데이터 소스 정의
    datasource_definition = {
        "name": f"Azure Monitor - {MONITOR_WORKSPACE_NAME}",
        "type": "prometheus",
        "access": "proxy",
        "url": PROMETHEUS_QUERY_ENDPOINT,
        "jsonData": {
            "httpMethod": "POST",
            "azureCredentials": {
                "authType": "msi"
            }
        },
        "isDefault": True
    }
    
    # 데이터 소스 생성
    create_ds_result = subprocess.run([
        "az", "grafana", "data-source", "create",
        "--name", GRAFANA_NAME,
        "--resource-group", RESOURCE_GROUP,
        "--definition", json.dumps(datasource_definition)
    ], capture_output=True, text=True)
    
    if create_ds_result.returncode == 0:
        print("   ✅ Grafana → Prometheus 데이터 소스 연결 완료!")
        print(f"   📊 데이터 소스 이름: Azure Monitor - {MONITOR_WORKSPACE_NAME}")
        print(f"   🔗 Prometheus Endpoint: {PROMETHEUS_QUERY_ENDPOINT}")
        print()
        print("   💡 이제 Grafana에서:")
        print("      - Prometheus 메트릭 쿼리 가능")
        print("      - Kubernetes 대시보드 사용 가능")
        print("      - 커스텀 대시보드 생성 가능")
    else:
        print("   ⚠️  데이터 소스 추가 중 오류")
        if create_ds_result.stderr:
            print(f"   {create_ds_result.stderr}")
        print()
        print("   💡 수동 추가 방법:")
        print("      1. Grafana 접속 → Configuration → Data sources")
        print("      2. Add data source → Prometheus 선택")
        print(f"      3. URL: {PROMETHEUS_QUERY_ENDPOINT}")
        print("      4. Auth: Azure Managed Identity")
else:
    print("   ✅ Prometheus 데이터 소스가 이미 연결되어 있습니다")

print()
print("✅ Part 2 완료: Grafana 설정 및 Prometheus 데이터 소스 연결 완료")
print("=" * 80)

🔧 Part 2: Grafana 설정 및 Prometheus 데이터 소스 연결

📊 구성할 Grafana 스택:
   1. Azure Managed Grafana (시각화 도구)
   2. Grafana가 Prometheus 데이터를 조회하도록 데이터 소스 연결 ⭐

1️⃣ 관리형 Grafana 워크스페이스 생성
   Grafana 이름: aks-mini-labs-grafana
   리소스 그룹: aks-mini-labs-rg

✅ Grafana 워크스페이스 생성 완료: aks-mini-labs-grafana

   🔍 Grafana 워크스페이스 상태 확인 중...
   ✅ Grafana ID: /subscriptions/8627ae60-01d3-4a2d-9c33-89dea54cd4b4/resourceGroups/aks-mini-labs-rg/providers/Microsoft.Dashboard/grafana/aks-mini-labs-grafana
   ✅ Grafana URL: https://aks-mini-labs-grafana-h5fkgcc9d4fddzc6.sel.grafana.azure.com
   ✅ 상태: Succeeded

2️⃣ AKS에 Grafana 연결 설정
   💡 AKS가 이 Grafana를 사용하도록 연결
   💡 중요: Prometheus 메트릭과 함께 설정해야 grafanaResourceId가 적용됩니다!
   ⏳ 설정 중... (2-3분 소요)

   ⏳ 연결 상태 확인 중... (10초 대기)
   ⚠️  설정 명령은 완료되었으나 상태 확인 실패
   💡 5-10분 후 Portal에서 확인 필요

3️⃣ Grafana가 Prometheus 데이터를 조회하도록 데이터 소스 연결 ⭐
   💡 이것이 진짜 Prometheus와 Grafana 연결입니다!
   💡 Grafana가 Prometheus 메트릭을 읽을 수 있게 설정

   🔍 기존 Prometheus 데이터 소스 확인 중...
   ⏳ Prometheus 데이터 소스 추가 중

### 5.2.2 Azure Portal에서 Grafana 연결 (중요!)

Grafana 리소스 생성 후, **Azure Portal에서 AKS와 Grafana를 수동으로 연결**해야 합니다. 이 단계는 Azure CLI로 완벽하게 작동하지 않으므로 Portal GUI 방식을 사용합니다.

#### 🔗 Portal 연결 단계 (정확한 경로):

1. **Azure Portal 접속**
   - 브라우저에서 [https://portal.azure.com](https://portal.azure.com) 열기

2. **AKS 클러스터로 이동**
   - 상단 검색창에 **`aks-mini-labs-cluster`** 입력
   - "Resources" 섹션에서 해당 클러스터 클릭

3. **Monitor settings 열기**
   - 왼쪽 메뉴에서 **"Monitoring"** 섹션 찾기
   - **"Insights"** 클릭
   - 화면 상단의 **"Monitor settings"** 버튼 클릭 (톱니바퀴 아이콘 옆)

4. **Grafana 연결 (핵심 단계!)**
   - "Managed Grafana" 섹션을 찾습니다
   - **"Link Grafana workspace"** 드롭다운 클릭
   - 목록에서 **`aks-mini-labs-grafana`** 선택
   - 화면 하단의 **"Configure"** 버튼 클릭

5. **연결 완료 대기**
   - 연결 작업은 약 **2-5분** 소요
   - 성공 시 녹색 알림 메시지 표시: "Successfully configured monitoring"
   - 새로고침하면 "Grafana workspace: aks-mini-labs-grafana" 표시 확인

#### ⚠️ 문제 해결

**드롭다운에 Grafana가 보이지 않는 경우:**
- Portal 페이지를 **새로고침(F5)** 후 다시 시도
- 이전 셀에서 RBAC 권한을 먼저 설정했는지 확인
- 권한 적용까지 1-2분 대기 후 재시도

**"You do not have access to the associated data collection rule" 오류:**
- RBAC 권한 설정 셀을 실행했는지 확인 (특히 DCR 권한)
- 권한 적용 대기 후 페이지 새로고침

#### ⏱️ 대시보드 자동 생성 대기

연결 완료 후 **5-10분** 정도 기다리면 Grafana에 Kubernetes 대시보드가 자동으로 생성됩니다:

- **Kubernetes / Compute Resources / Cluster**
- **Kubernetes / Compute Resources / Namespace (Pods)**
- **Kubernetes / Compute Resources / Node (Pods)**
- **Kubernetes / Compute Resources / Pod**
- **Kubernetes / Compute Resources / Workload**

### 5.2.2 Container Insights 설정

Log Analytics Workspace를 생성하고 AKS 클러스터에 Container Insights addon을 활성화합니다. Container Insights는 컨테이너 로그, 성능 메트릭, 인벤토리 데이터를 수집하여 Azure Portal에서 조회할 수 있도록 합니다.

**주요 작업:**
- Log Analytics Workspace 생성 (로그 저장소)
- Container Insights addon 활성화 상태 확인
- Container Insights addon 활성화 (필요 시)

In [45]:
import subprocess
import warnings
import json
import time

# Azure CLI 확장의 pkg_resources 경고 메시지 숨기기
warnings.filterwarnings('ignore', message='.*pkg_resources.*')

print("🔧 Part 2: Container Insights 활성화")
print("=" * 80)
print()
print("📊 구성할 항목:")
print("   1. Log Analytics Workspace (컨테이너 로그 저장소)")
print("   2. Container Insights Addon (로그 수집 에이전트)")
print("=" * 80)
print()

# Log Analytics 워크스페이스 이름 설정
LOG_WORKSPACE_NAME = f"{AKS_CLUSTER_NAME}-logs"

# Log Analytics 워크스페이스 생성 (또는 기존 것 사용)
print(f"1️⃣ Log Analytics 워크스페이스 생성")
print(f"   워크스페이스 이름: {LOG_WORKSPACE_NAME}")
print(f"   리소스 그룹: {RESOURCE_GROUP}")
print()

log_ws_result = subprocess.run([
    "az", "monitor", "log-analytics", "workspace", "create",
    "--resource-group", RESOURCE_GROUP,
    "--workspace-name", LOG_WORKSPACE_NAME,
    "--location", LOCATION
], capture_output=True, text=True)

if log_ws_result.returncode == 0 or "already exists" in log_ws_result.stderr.lower():
    if log_ws_result.returncode == 0:
        print("✅ Log Analytics 워크스페이스 생성 완료")
    else:
        print("✅ Log Analytics 워크스페이스가 이미 존재합니다")
    
    # Log Analytics 워크스페이스 ID 가져오기
    log_ws_id_result = subprocess.run([
        "az", "monitor", "log-analytics", "workspace", "show",
        "--resource-group", RESOURCE_GROUP,
        "--workspace-name", LOG_WORKSPACE_NAME,
        "--query", "id",
        "-o", "tsv"
    ], capture_output=True, text=True)
    
    if log_ws_id_result.returncode == 0:
        LOG_WS_ID = log_ws_id_result.stdout.strip()
        print(f"✅ 워크스페이스 ID: {LOG_WS_ID}")
        print()
        
        # Container Insights가 이미 활성화되어 있는지 먼저 확인
        print("2️⃣ Container Insights 상태 확인")
        ci_check = subprocess.run([
            "az", "aks", "show",
            "--name", AKS_CLUSTER_NAME,
            "--resource-group", RESOURCE_GROUP,
            "--query", "addonProfiles.omsagent.enabled",
            "-o", "tsv"
        ], capture_output=True, text=True)
        
        is_enabled = ci_check.returncode == 0 and ci_check.stdout.strip().lower() == 'true'
        
        if is_enabled:
            print("✅ Container Insights가 이미 활성화되어 있습니다")
            print("📊 컨테이너 로그 수집 중")
        else:
            print("⏳ Container Insights 활성화 중...")
            print()
            print("   Log Analytics 워크스페이스 초기화 대기 중 (3분)...")
            print("   💡 필수 테이블이 생성될 때까지 대기합니다.")
            time.sleep(180)  # 3분 대기
            
            # Container Insights 활성화
            print("   ⏳ Container Insights addon 활성화 중...")
            insights_result = subprocess.run([
                "az", "aks", "enable-addons",
                "--addon", "monitoring",
                "--resource-group", RESOURCE_GROUP,
                "--name", AKS_CLUSTER_NAME,
                "--workspace-resource-id", LOG_WS_ID
            ], capture_output=True, text=True)
            
            if insights_result.returncode == 0:
                print("   ✅ Container Insights 활성화 완료")
                print("   📊 컨테이너 로그 수집 시작")
            else:
                print("   ⚠️ Container Insights 활성화 중 오류")
                if insights_result.stderr:
                    print(f"   {insights_result.stderr}")
                print()
                print("   💡 해결 방법:")
                print("      1. 이미 활성화되어 있을 수 있음 (위 상태 확인)")
                print("      2. Azure Portal → AKS → Insights에서 상태 확인")
    else:
        print("❌ 워크스페이스 ID 조회 실패")
        if log_ws_id_result.stderr:
            print(f"   {log_ws_id_result.stderr}")
else:
    print("⚠️ Log Analytics 워크스페이스 생성 중 오류")
    if log_ws_result.stderr:
        print(f"   {log_ws_result.stderr}")

print()
print("✅ Part 2 완료: Container Insights 설정 완료")
print("=" * 80)


🔧 Part 2: Container Insights 활성화

📊 구성할 항목:
   1. Log Analytics Workspace (컨테이너 로그 저장소)
   2. Container Insights Addon (로그 수집 에이전트)

1️⃣ Log Analytics 워크스페이스 생성
   워크스페이스 이름: aks-mini-labs-cluster-logs
   리소스 그룹: aks-mini-labs-rg

✅ Log Analytics 워크스페이스 생성 완료
✅ 워크스페이스 ID: /subscriptions/8627ae60-01d3-4a2d-9c33-89dea54cd4b4/resourceGroups/aks-mini-labs-rg/providers/Microsoft.OperationalInsights/workspaces/aks-mini-labs-cluster-logs

2️⃣ Container Insights 상태 확인
⏳ Container Insights 활성화 중...

   Log Analytics 워크스페이스 초기화 대기 중 (3분)...
   💡 필수 테이블이 생성될 때까지 대기합니다.
   ⏳ Container Insights addon 활성화 중...
   ✅ Container Insights 활성화 완료
   📊 컨테이너 로그 수집 시작

✅ Part 2 완료: Container Insights 설정 완료


### 5.3 로그 진단 (Log Diagnostics)

Container Insights를 통해 수집된 로그는 Log Analytics Workspace에 저장되며, KQL(Kusto Query Language)을 사용하여 다양한 방식으로 조회하고 분석할 수 있습니다. 이 섹션에서는 주요 로그 테이블과 실전 진단 쿼리를 다룹니다.

#### 📊 Prometheus vs Container Insights vs Diagnostic Settings: 차이점 이해하기

앞서 설정한 세 가지 모니터링 솔루션의 목적과 역할이 다릅니다:

| 구분 | Prometheus (Metrics) | Container Insights (Data Plane Logs) | Diagnostic Settings (Control Plane Logs) |
|------|----------------------|--------------------------------------|------------------------------------------|
| **데이터 유형** | 📈 메트릭 (Metrics) | 📝 데이터 플레인 로그 | 🔧 제어 플레인 로그 |
| **수집 내용** | CPU, Memory, Network 등 **수치 데이터** | stdout/stderr, 이벤트, 인벤토리 등 **컨테이너 텍스트 데이터** | API Server, Audit, Scheduler 등 **Kubernetes 제어 로그** |
| **저장소** | Azure Monitor Workspace | Log Analytics Workspace | Log Analytics Workspace (AzureDiagnostics 테이블) |
| **쿼리 언어** | PromQL | KQL (Kusto Query Language) | KQL (Kusto Query Language) |
| **시각화** | Grafana 대시보드 | Azure Workbooks, Portal | Azure Workbooks, Portal |
| **용도** | 성능 모니터링, 실시간 대시보드, 알람 | 애플리케이션 로그 분석, 디버깅, 근본 원인 분석 | 보안 감사, 클러스터 관리 작업 추적 |
| **예시 질문** | "지금 CPU 사용률은?" "메모리 추이는?" | "왜 에러가 났지?" "어떤 로그가 있었지?" | "누가 무엇을 변경했지?" "API 호출 실패 원인은?" |

**💡 실무에서의 활용:**
- **Prometheus/Grafana**: "지금 시스템 상태가 어떤가?" (실시간 성능 모니터링)
- **Container Insights**: "왜 30분 전에 Pod가 재시작됐지?" (애플리케이션 사후 분석)
- **Diagnostic Settings**: "누가 클러스터 설정을 변경했지?" (보안 감사 및 규정 준수)

세 솔루션은 **상호 보완적**이며, 완전한 관찰성(Observability)을 위해 함께 사용하는 것이 권장됩니다.

---

**주요 로그 테이블:**
- `ContainerLogV2`: 컨테이너 표준 출력/오류 로그 (stdout/stderr)
- `KubePodInventory`: Pod 인벤토리 및 상태 정보
- `KubeEvents`: Kubernetes 이벤트 로그
- `Perf`: 성능 카운터 (CPU, Memory 사용량)
- `KubeNodeInventory`: Node 인벤토리 및 상태 정보

**로그 진단 사용 시나리오:**
1. 애플리케이션 오류 추적
2. Pod Restart/CrashLoopBackOff 원인 분석
3. 리소스 부족 문제 진단
4. 네트워크 연결 문제 디버깅
5. 성능 병목 지점 파악

#### 5.3.0 진단 설정 구성 (Diagnostic Settings)

Container Insights가 활성화된 후, **진단 설정(Diagnostic Settings)**을 구성하여 추가 로그 카테고리를 수집할 수 있습니다. 기본 Container Insights는 컨테이너 로그와 성능 데이터를 수집하지만, Kubernetes API 서버 로그, 감사 로그 등의 제어 영역(Control Plane) 로그는 별도 설정이 필요합니다.

**수집 가능한 로그 카테고리:**
- **kube-apiserver**: Kubernetes API 서버 로그 (클러스터 관리 작업 추적)
- **kube-audit**: 감사 로그 (보안 감사 및 규정 준수)
- **kube-controller-manager**: 컨트롤러 매니저 로그
- **kube-scheduler**: 스케줄러 로그 (Pod 배치 결정)
- **cluster-autoscaler**: 클러스터 오토스케일러 로그
- **guard**: 인증/인가 관련 로그

이 설정은 **보안 감사, 규정 준수, 고급 트러블슈팅**에 필수적입니다.

In [46]:
import subprocess
import warnings

warnings.filterwarnings('ignore', message='.*pkg_resources.*')

print("🔧 AKS 진단 설정 구성")
print("=" * 80)
print()

# 1. 현재 진단 설정 확인
print("1️⃣ 현재 진단 설정 확인")
diag_check = subprocess.run([
    "az", "monitor", "diagnostic-settings", "list",
    "--resource", f"/subscriptions/{subprocess.run(['az', 'account', 'show', '--query', 'id', '-o', 'tsv'], capture_output=True, text=True).stdout.strip()}/resourceGroups/{RESOURCE_GROUP}/providers/Microsoft.ContainerService/managedClusters/{AKS_CLUSTER_NAME}",
    "--query", "[].{name:name, logs:logs[?enabled].category}",
    "-o", "table"
], capture_output=True, text=True)

if diag_check.returncode == 0:
    print(diag_check.stdout)
    if "No diagnostic settings" in diag_check.stdout or len(diag_check.stdout.strip()) < 50:
        print("   ℹ️  진단 설정이 구성되지 않았습니다.")
    else:
        print("   ✅ 진단 설정이 구성되어 있습니다.")
else:
    print("   ⚠️ 확인 실패")

print()

# 2. 진단 설정 생성 (제어 영역 로그)
print("2️⃣ 진단 설정 생성")
print("   Control Plane 로그를 Log Analytics로 전송 설정 중...")
print()

# AKS 리소스 ID 가져오기
aks_id = subprocess.run([
    "az", "aks", "show",
    "--name", AKS_CLUSTER_NAME,
    "--resource-group", RESOURCE_GROUP,
    "--query", "id",
    "-o", "tsv"
], capture_output=True, text=True).stdout.strip()

# 진단 설정 생성
diag_name = f"{AKS_CLUSTER_NAME}-diagnostics"

diag_create = subprocess.run([
    "az", "monitor", "diagnostic-settings", "create",
    "--name", diag_name,
    "--resource", aks_id,
    "--workspace", LOG_WS_ID,
    "--logs", '[{"category":"kube-apiserver","enabled":true},{"category":"kube-audit","enabled":true},{"category":"kube-controller-manager","enabled":true},{"category":"kube-scheduler","enabled":true},{"category":"cluster-autoscaler","enabled":true}]',
    "--metrics", '[{"category":"AllMetrics","enabled":true}]'
], capture_output=True, text=True)

if diag_create.returncode == 0:
    print("   ✅ 진단 설정 생성 완료!")
    print()
    print("   📊 활성화된 로그 카테고리:")
    print("      - kube-apiserver: API 서버 요청/응답 로그")
    print("      - kube-audit: 클러스터 감사 로그 (보안)")
    print("      - kube-controller-manager: 컨트롤러 로그")
    print("      - kube-scheduler: Pod 스케줄링 로그")
    print("      - cluster-autoscaler: 오토스케일러 로그")
    print("      - AllMetrics: 제어 영역 메트릭")
elif "already exists" in diag_create.stderr.lower() or "conflict" in diag_create.stderr.lower():
    print("   ✅ 진단 설정이 이미 존재합니다.")
    print()
    print("   💡 기존 설정을 업데이트하려면:")
    print("      1. Azure Portal → AKS 클러스터 → Diagnostic settings")
    print("      2. 기존 설정 클릭 → 로그 카테고리 수정 → Save")
else:
    print("   ⚠️ 진단 설정 생성 중 오류")
    if diag_create.stderr:
        print(f"   {diag_create.stderr}")
    print()
    print("   💡 대안: Azure Portal에서 수동 설정")
    print("      1. Azure Portal → AKS 클러스터 → Diagnostic settings")
    print("      2. 'Add diagnostic setting' 클릭")
    print("      3. Log Analytics workspace 선택")
    print("      4. 원하는 로그 카테고리 선택 → Save")

print()
print("=" * 80)
print("✅ 진단 설정 구성 완료")
print()
print("💡 다음 단계:")
print("   - 로그 수집까지 5-10분 소요")
print("   - Azure Portal → Log Analytics Workspace → Logs에서 쿼리 가능")
print("   - 다음 셀들에서 KQL 쿼리로 로그 분석")
print("=" * 80)

🔧 AKS 진단 설정 구성

1️⃣ 현재 진단 설정 확인


   ℹ️  진단 설정이 구성되지 않았습니다.

2️⃣ 진단 설정 생성
   Control Plane 로그를 Log Analytics로 전송 설정 중...

   ✅ 진단 설정 생성 완료!

   📊 활성화된 로그 카테고리:
      - kube-apiserver: API 서버 요청/응답 로그
      - kube-audit: 클러스터 감사 로그 (보안)
      - kube-controller-manager: 컨트롤러 로그
      - kube-scheduler: Pod 스케줄링 로그
      - cluster-autoscaler: 오토스케일러 로그
      - AllMetrics: 제어 영역 메트릭

✅ 진단 설정 구성 완료

💡 다음 단계:
   - 로그 수집까지 5-10분 소요
   - Azure Portal → Log Analytics Workspace → Logs에서 쿼리 가능
   - 다음 셀들에서 KQL 쿼리로 로그 분석


## 6. Node Auto Provisioning(노드 자동 프로비저닝) 적용

Node Auto Provisioning(NAP)은 AKS에서 워크로드 수요에 따라 다양한 VM 크기의 노드 풀을 자동으로 생성/확장/축소하는 기능입니다. 비용 최적화와 리소스 효율성을 위해 권장되는 Azure 기능입니다.

### 6.1 사전 준비

NAP를 사용하기 위해서는 먼저 필요한 Azure CLI 확장을 설치하고 기능 플래그를 등록해야 합니다.

In [ ]:
import subprocess

print("📦 aks-preview 확장 설치/업데이트 중...")
print()

# aks-preview 확장 설치 또는 업데이트
result = subprocess.run([
    "az", "extension", "add",
    "--name", "aks-preview",
    "--yes"
], capture_output=True, text=True)

if "already installed" in result.stderr.lower() or result.returncode == 0:
    print("✅ aks-preview 확장이 설치되어 있습니다. 최신 버전으로 업데이트 중...")
    subprocess.run([
        "az", "extension", "update",
        "--name", "aks-preview"
    ])
else:
    print("✅ aks-preview 확장 설치 완료")

print()
print("🔧 NodeAutoProvisioningPreview Feature Flag 등록 중...")
print()

# Feature Flag 등록
subprocess.run([
    "az", "feature", "register",
    "--namespace", "Microsoft.ContainerService",
    "--name", "NodeAutoProvisioningPreview"
])

print()
print("📋 Feature Flag 등록 상태 확인:")
subprocess.run([
    "az", "feature", "show",
    "--namespace", "Microsoft.ContainerService",
    "--name", "NodeAutoProvisioningPreview",
    "--output", "table"
])

print()
print("💡 참고:")
print("   - Feature Flag가 'Registered' 상태가 될 때까지 몇 분이 걸릴 수 있습니다.")
print("   - 'Registered' 상태 확인 후 다음 셀을 실행하세요.")

### 6.2 Provider 재등록

Feature Flag가 'Registered' 상태가 되면 Provider를 재등록해야 합니다.

In [ ]:
import subprocess

print("🔄 Microsoft.ContainerService Provider 재등록 중...")
print()

result = subprocess.run([
    "az", "provider", "register",
    "--namespace", "Microsoft.ContainerService"
])

if result.returncode == 0:
    print("✅ Provider 재등록 완료")
    print()
    print("💡 다음 단계:")
    print("   몇 분 후 섹션 6.3을 실행하여 NAP를 활성화하세요.")
else:
    print("❌ Provider 재등록 실패")

### 6.3 기존 클러스터에 NAP 활성화

기존 AKS 클러스터에 Node Auto Provisioning을 활성화합니다.

> **중요**: NAP를 활성화하면 기존 수동 노드 풀은 유지되지만, 향후 자동 프로비저닝된 노드 풀이 추가로 생성됩니다.

In [ ]:
import subprocess

print(f"🚀 기존 클러스터에 NAP 활성화 중: {AKS_CLUSTER_NAME}")
print("⏱️  예상 소요 시간: 5-10분")
print()

# NAP 활성화
result = subprocess.run([
    "az", "aks", "update",
    "--resource-group", RESOURCE_GROUP,
    "--name", AKS_CLUSTER_NAME,
    "--node-provisioning-mode", "Auto"
], capture_output=True, text=True)

if result.returncode == 0:
    print("✅ NAP 활성화 완료!")
    print()
    print("📊 클러스터 상태 확인:")
    subprocess.run([
        "az", "aks", "show",
        "--resource-group", RESOURCE_GROUP,
        "--name", AKS_CLUSTER_NAME,
        "--query", "{Name:name, NodeProvisioningMode:nodeProvisioningProfile.mode}",
        "--output", "table"
    ])
else:
    print("❌ NAP 활성화 실패")
    print(result.stderr)

print()
print("💡 NAP 동작 방식:")
print("   - Pod가 스케줄될 수 없는 상황에서 AKS가 자동으로 적절한 VM 크기의 노드 풀을 생성")
print("   - 워크로드가 줄어들면 불필요한 노드 풀이 자동으로 축소/삭제")
print("   - Spot VM, GPU VM 등 다양한 VM 옵션을 자동으로 활용")

### 6.4 NAP 테스트: 워크로드 증가

NAP가 정상적으로 동작하는지 테스트하기 위해 Pod 수를 대폭 늘려 노드 부족 상황을 유도합니다.

> **참고**: HPA(Horizontal Pod Autoscaler)가 설정되어 있으면 수동 스케일링을 제한할 수 있으므로, 이 셀에서는 HPA를 먼저 삭제합니다.

In [ ]:
import subprocess

print("🔥 워크로드 대폭 증가 - NAP 테스트 시작")
print()

# HPA가 있으면 삭제 (수동 스케일링을 위해)
print("1️⃣ HPA 확인 및 삭제:")
hpa_check = subprocess.run([
    "kubectl", "get", "hpa", "myapp"
], capture_output=True, text=True)

if hpa_check.returncode == 0:
    print("   HPA가 존재합니다. NAP 테스트를 위해 삭제합니다...")
    subprocess.run(["kubectl", "delete", "hpa", "myapp"])
    print("   ✅ HPA 삭제 완료")
else:
    print("   ✅ HPA가 없습니다. 계속 진행합니다.")

print()

# 현재 노드 상태 확인
print("2️⃣ 현재 노드 상태:")
subprocess.run(["kubectl", "get", "nodes"])

print()
print("3️⃣ 현재 노드 풀 목록:")
subprocess.run([
    "az", "aks", "nodepool", "list",
    "--resource-group", RESOURCE_GROUP,
    "--cluster-name", AKS_CLUSTER_NAME,
    "--query", "[].{Name:name, Count:count, Mode:mode}",
    "--output", "table"
])

print()
print("4️⃣ Pod 수를 100개로 증가 (NAP 트리거)...")
subprocess.run([
    "kubectl", "scale", "deployment", "myapp",
    "--replicas=100"
])

print()
print("⏱️  잠시 대기 중 (Pod 생성 대기)...")
subprocess.run(["sleep", "5"])

print()
print("5️⃣ Pod 상태 확인:")
subprocess.run(["kubectl", "get", "pods", "-o", "wide"])

print()
print("6️⃣ Pending 상태 Pod 확인:")
pending_result = subprocess.run([
    "kubectl", "get", "pods",
    "--field-selector=status.phase=Pending"
], capture_output=True, text=True)

if "No resources found" not in pending_result.stdout:
    print(pending_result.stdout)
    print("⚠️  Pending 상태의 Pod가 있습니다!")
    print("   NAP가 자동으로 새 노드를 추가할 것입니다.")
else:
    print("   ✅ 모든 Pod가 Running 상태입니다.")
    print("   현재 노드 리소스가 충분합니다.")

print()
print("=" * 80)
print("💡 다음 단계:")
print("   1. 몇 분 후 섹션 6.5를 실행하여 NAP가 새 노드 풀을 생성했는지 확인")
print("   2. Pending 상태의 Pod가 있으면 NAP가 자동으로 노드를 추가합니다")
print("   3. NAP가 트리거되지 않았다면 현재 노드 용량이 충분한 것입니다")
print("=" * 80)


### 6.5 NAP 동작 확인

NAP에 의해 자동으로 생성된 노드 풀과 노드를 확인합니다.

In [ ]:
import subprocess

print("🔍 NAP 동작 확인 - 자동 생성된 노드 풀 및 노드 조회")
print()

print("📊 현재 노드 풀 목록 (NAP에 의해 생성된 풀 포함):")
subprocess.run([
    "az", "aks", "nodepool", "list",
    "--resource-group", RESOURCE_GROUP,
    "--cluster-name", AKS_CLUSTER_NAME,
    "--output", "table"
])

print()
print("📊 현재 노드 상태:")
subprocess.run(["kubectl", "get", "nodes", "-o", "wide"])

print()
print("📊 Pod 분산 상태:")
subprocess.run([
    "kubectl", "get", "pods",
    "-o", "custom-columns=NAME:.metadata.name,STATUS:.status.phase,NODE:.spec.nodeName"
])

print()
print("💡 확인 사항:")
print("   ✅ 새로운 노드 풀이 자동 생성되었는지 확인")
print("   ✅ Pending 상태였던 Pod들이 새 노드에 스케줄링되었는지 확인")
print("   ✅ NAP가 워크로드에 맞는 적절한 VM 크기를 선택했는지 확인")

### 6.6 NAP 테스트: 워크로드 감소

워크로드를 줄여서 NAP가 자동으로 노드 풀을 축소/삭제하는지 확인합니다.

In [ ]:
import subprocess

print("📉 워크로드 감소 - NAP 자동 축소 테스트")
print()

print("🔄 Pod 수를 다시 2개로 축소...")
subprocess.run([
    "kubectl", "scale", "deployment", "myapp",
    "--replicas=2"
])

print()
print("📊 현재 Pod 상태:")
subprocess.run(["kubectl", "get", "pods"])

print()
print("💡 NAP 자동 축소 확인 방법:")
print("   1. 몇 분 후 다시 노드 풀 목록을 확인")
print("   2. 불필요한 노드 풀이 자동으로 삭제되는지 모니터링")
print()
print("   실시간 모니터링 명령:")
print("   kubectl get nodes --watch")
print("   az aks nodepool list --resource-group", RESOURCE_GROUP, "--cluster-name", AKS_CLUSTER_NAME)

### 6.7 NAP 장점 정리

Node Auto Provisioning을 사용하면 다음과 같은 이점을 얻을 수 있습니다:

- ✅ **비용 최적화**: 워크로드에 따라 자동으로 스케일링되어 불필요한 리소스 비용 절감
- ✅ **리소스 효율성**: 다양한 VM 크기를 자동으로 선택하여 최적의 리소스 활용
- ✅ **운영 자동화**: 수동 노드 풀 관리 없이 자동으로 확장/축소
- ✅ **유연성**: GPU, Spot VM 등 다양한 VM 옵션 자동 활용
- ✅ **고가용성**: Pod가 Pending 상태에 머무르지 않고 즉시 스케줄링

> **참고**: 자세한 내용은 [Azure 공식 문서](https://learn.microsoft.com/en-us/azure/aks/node-autoprovision?tabs=azure-cli)를 참조하세요.

## 7. 정리

AKS 실습을 통해 클러스터 생성, 모니터링, 오토스케일링, Node Auto Provisioning 등 Kubernetes 기반 애플리케이션의 배포와 운영 자동화 전 과정을 GitHub Codespaces 환경에서 단계별로 실습했습니다.

### 실습한 내용 요약
- ✅ AKS 클러스터 생성 및 구성
- ✅ ACR 연동 및 이미지 배포
- ✅ LoadBalancer Service를 통한 외부 노출
- ✅ 수동 및 자동 스케일링 (HPA)
- ✅ Azure Monitor, Prometheus, Grafana를 통한 모니터링
- ✅ Node Auto Provisioning 테스트

> **다음 단계**: `03-cicd-automation.ipynb`에서 GitHub Actions와 Azure Pipelines를 사용한 CI/CD 자동화를 실습합니다.